## Assignment 1: Build a Predictor
## Dhruval Bhatt (Collaborated with Keertana Chidambaram)

Instructions from assignment:

Build a predictor . Once you have built a predictor, you should use the test data set to submit a
csv file on canvas. That csv file should only have two columns: diag_id, y_hat.

y_hat = drinks per day = 'U1031900'
diag_id = 'diag.id'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# Load the data given

train_df = pd.read_csv('../Data/nlsy training set.csv')
test_df = pd.read_csv('../Data/nlsy test set.csv')

print("Size of Training Set", train_df.shape)
print("Size of Test Set", test_df.shape)

train_df.drop(train_df.columns[[0]], axis=1, inplace=True)  #remove unnamed column that is the repeat of id column in train
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)#remove unnamed column that is the repeat of id column in test

print("Size of Training Set after Removing Col 1", train_df.shape)
print("Size of Test Set after Removing Col 1", test_df.shape)

Size of Training Set (7187, 4887)
Size of Test Set (1797, 4886)
Size of Training Set after Removing Col 1 (7187, 4886)
Size of Test Set after Removing Col 1 (1797, 4885)


In [3]:
train_df.set_index('diag.id', inplace=True)

### Data Prep

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Remove rows that have missing values for the target data
train_df = train_df[train_df['U1031900'] >= 0] #subset

In [6]:
train_X = train_df.loc[:, train_df.columns != 'U1031900']
print("Size of Training Set X variables", train_X.shape)

Size of Training Set X variables (3545, 4884)


In [7]:
train_y = train_df['U1031900']
print("Size of Training Set y variables", train_y.shape)

Size of Training Set y variables (3545,)


In [8]:
# Split into train and test 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=0)

In [9]:
X_train.shape

(2836, 4884)

In [24]:
y_test.shape

(709,)

### Feature Selection 

In [10]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer
import numpy

C:\Users\Dhruval\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
# Decision Tree

model_1 = tree.DecisionTreeClassifier()
model_1.fit(X_train, y_train)
pred_1 = model_1.predict(X_test)
mse_1 = mean_squared_error(y_test, pred_1)
print('MSE: %f' % mse_1)

MSE: 29.873061


In [12]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model_1.feature_importances_, index= X_train.columns)
#feat_importances.nlargest(10).plot(kind='barh')
#plt.show()

selected_features_1 = list(feat_importances.nlargest(2000).index)

In [13]:
# Decision Tree with Selected (Just kept the top 2000 predictors)

X_train_sel = X_train.filter(selected_features_1)
X_test_sel = X_test.filter(selected_features_1)

model_2 = tree.DecisionTreeClassifier()
model_2.fit(X_train_sel, y_train)
pred_2 = model_2.predict(X_test_sel)
mse_2 = mean_squared_error(y_test, pred_2)
print('MSE: %f' % mse_2)

MSE: 25.521862


In [14]:
#Feature selection with Lasso Regression


In [15]:
# Random Forest Decision Tree

rf_c = RandomForestClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=10, 
                                 n_estimators=25, max_features=4000, random_state=0)
rf_c.fit(X_train, y_train)
pred_rfc = rf_c.predict(X_test)

mean_squared_error(y_test, pred_rfc)

18.41184767277856

In [16]:
#XGB Regressor
# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 10)
xgb_r = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 15, random_state=0)
xgb_r.fit(X_train,y_train)
pred_xgbr = xgb_r.predict(X_test)
pred_xgbr = numpy.around(pred_xgbr)

mean_squared_error(y_test, pred_xgbr)

C:\Users\Dhruval\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


17.698166431593794

In [ ]:
#XGB Classifier
# xgb_c = xgb.XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='multi:softmax', 
#                       n_estimators=100, 
#                       reg_alpha = 0.3,
#                       max_depth=4, 
#                       gamma=10)

# # clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators = 2, max_depth  = 2)
# xgb_c.fit(X_train, y_train)
# preds_xgbc = xgb_c.predict(X_test)
# #yhat_c = np.around(preds)
# mean_squared_error(y_test, preds_xgbc)

In [ ]:
#Ensemble Methods
# yhat_rfc = RFC_model.predict(X_test)
# yhat_XGBR = XGBR_model.predict(X_test)
# yhat_XGBC = clf.predict(X_test)

# #yhat_df = pd.concat([yhat_rfc, yhat_XGBR], axis=1, sort=False)
# yhat_rfc.shape

In [19]:
from sklearn.model_selection import StratifiedKFold

def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((test.shape[0],1),float)
    train_pred=np.empty((0,1),float)
    
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    
    return test_pred.reshape(-1,1),train_pred

In [20]:
test_pred1 ,train_pred1=Stacking(model=rf_c,n_fold=10, train=X_train,test=X_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

C:\Users\Dhruval\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [21]:
test_pred2 ,train_pred2=Stacking(model=xgb_r,n_fold=10,train=X_train,test=X_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

C:\Users\Dhruval\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Dhruval\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Dhruval\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Dhruval\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Dhruval\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be remove

In [28]:
df.shape

(2836, 2)

In [33]:
df_test.shape

(7799, 2)

In [35]:
from sklearn.linear_model import LogisticRegression

df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df, y_train)
pred = model.predict(df_test)
#mean_squared_error(y_test, pred)
model.score(df_test, pred)

1.0

In [36]:
pred.shape

(7799,)

In [ ]:
from scipy import stats

final_pred = np.array([])
for i in range(0,len(yhat_rfc)):
    final_pred = np.append(final_pred, stats.mode([yhat_rfc[i], yhat_XGBR[i], yhat_XGBC[i]]))

In [ ]:
## Apply Final Model to Test Data: 

y_hat_final = predict(test_df)

diag = pd.DataFrame(test_df.index)
predsc = pd.DataFrame(y_hat_final)
df = pd.concat([diag, predsc], axis=1, sort=False)